<a href="https://colab.research.google.com/github/marcelobenedito/quarantine_covid19_behavior_analysis/blob/master/quarantine_covid19_behavior_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Quarantine Covid-19 behavior analysis**

*It will be collect data tweets about COVID-19, quarantine and related about. This content will analysed to extract sentiment and main user behavior that makes don't stay home.*

## **1. Initial settings**

**Install libraries**

In [1]:
!pip3 install unidecode
!pip3 install emoji
!pip3 install joblib

     |████████████████████████████████| 245kB 5.3MB/s 
     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=6f2b679a93220c600e7f2a3e42b254dbf644cdb1daebf2920a7ea54ce93f3969
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


**Required imports**

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import string
import tweepy
import time
import json
import nltk
import re
from datetime import timedelta
from google.colab import drive
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from emoji import demojize
from pytz import timezone

**Mounting drive and downloading stop words**

In [3]:
drive.mount('/content/drive')
nltk.download('stopwords')

Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Add America/São Paulo timezone for this notebook**

In [4]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/America/Sao_Paulo /etc/localtime
!date

tz = timezone('America/Sao_Paulo')

Sun Oct  4 10:18:19 -03 2020


In [5]:
cd 'drive/My Drive/TCC/credentials'

/content/drive/My Drive/TCC/credentials


In [6]:
from credentials import *

## **2. Extract tweets**

In [ ]:
cd '../Tweets'

**Handling the rate limit using cursors**

In [ ]:
def limit_handled(cursor, finishes):
    while True:
        try:
            if finishes == True:
              return
            yield cursor.next()
        except (tweepy.RateLimitError, StopIteration):
            if finishes == True:
              return
            else:
              print('\r{} [tweepy.RateLimitError] Rate limit exception...'.format(dt.datetime.now().astimezone(tz)), end='')
              time.sleep(15*60)

**Create a funcion to search tweets**

In [ ]:
def search_tweets(search_filter, since, until, limit, language):
  finishes = False
  for tweet in tweepy.Cursor(api.search, q=search_filter, until=until, lang=language, tweet_mode="extended").items(limit):
    tweets.append(tweet)
    print('\r{} [INFO] {} collected tweets!'.format(dt.datetime.now().astimezone(tz), len(tweets)), end='');
    if limit != None and len(tweets) == limit:
      return tweets

**API authentication**

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

**Defining filters used in search**

In [ ]:
""" não estou saindo "não estou saindo" (quarentena OR covid) (#covid-19 OR #coronavírus OR #coronavirus OR #covid OR #quarentena) lang:pt until:2020-01-31 since:2020-01-01 -filter:replies """

contains_both_words = ''
exact_phrase = ''
#contains_any_words = '(quarentena OR covid OR coronavirus OR isolamento OR festa OR role OR evento OR balada OR sair OR saindo)'
contains_any_words = ('(quarentena OR covid OR covid-19 OR coronavirus OR coronavírus'
                      ' OR isolamento OR isolado OR isolar OR festa OR rolê OR role'
                      ' OR evento OR balada OR sair OR saindo OR indo OR vamos OR vou'
                      ' OR churrasco OR churras OR churrascão OR rodeio OR show'
                      ' OR viagem OR viajar)')
contains_any_hashtags = ''
no_retweet = ''
language = 'pt'
#since = dt.date(2020,1,1)
until = dt.date.today() - dt.timedelta(days=7)
since = 'since:2020-01-01'
#until = 'until:2020-07-01'
limit = 30000

search_filter = contains_both_words + ' ' + exact_phrase + ' ' + contains_any_words + ' ' + contains_any_hashtags + ' ' + since + no_retweet

**Extracting tweets based on search filter**

In [ ]:
tweets = []
started_at = dt.datetime.now()
print('{} [INFO] starting search...'.format(dt.datetime.now().astimezone(tz)))
tweets = search_tweets(search_filter, since, until, limit, language)
finished_at = dt.datetime.now()

2020-10-03 10:03:00.511076-03:00 [INFO] starting search...
2020-10-03 12:49:05.846528-03:00 [INFO] 30000 collected tweets!

**Transform Json to DataFrame and export to CSV file**

In [ ]:
df = pd.DataFrame({
      'created_at': tweet.created_at,
      'user_id': tweet.user.id, 
      'screen_name': tweet.user.screen_name,
      'geo': tweet.geo,
      'coordinates': tweet.coordinates,
      'place': tweet.place,
      'tweet_id': tweet.id, 
      'text': unidecode(tweet.full_text),
      'class': ''
  } for tweet in tweets)

df.to_csv('tweets_{}.csv'.format(dt.datetime.now().astimezone(tz)), encoding = 'utf-8', index = False)

**Printing found tweets**

In [ ]:
df.head()

,created_at,user_id,screen_name,geo,coordinates,place,tweet_id,text,class
0,2020-09-25 23:59:59,1207713857310986242,taeminfoxy,None,None,None,1309643847857827841,"@euphoriajeonjkk vou fazer da jisoo, mas ele v...",
1,2020-09-25 23:59:59,1201458114161238016,patty_harleyy,None,None,None,1309643847731998720,@3LsApaixonada Vou pedir a ela amg,
2,2020-09-25 23:59:59,1184455667794493441,dutrinhamari,None,None,None,1309643847425814531,RT @luisasonza: e vou usar amanha dnv KKKKK gn...,
3,2020-09-25 23:59:59,3394646543,zayoungie,None,None,None,1309643847404793858,perai que eu vou procurar e ficar triste \n#Ja...,
4,2020-09-25 23:59:59,759844528895340544,VictoriaHGonz,None,None,None,1309643847392276480,"abrindo minha cerveja, pois vou estudar classi...",


**Write to json file**

In [ ]:
df = pd.DataFrame(tweets)
df.to_json('tweets_{}.json'.format(dt.datetime.now().astimezone(tz)))
df.head()

,0
0,Status(_api=<tweepy.api.API object at 0x7f3820...
1,Status(_api=<tweepy.api.API object at 0x7f3820...
2,Status(_api=<tweepy.api.API object at 0x7f3820...
3,Status(_api=<tweepy.api.API object at 0x7f3820...
4,Status(_api=<tweepy.api.API object at 0x7f3820...


## **3. Preprocessing**

This process is used to preprocess the tweet text:

 - Tokenize words;
 - Remove all stop words; 
 - Punctuaction rules; 
 - Unused characters;
 - Links from tweets.

In [55]:
cd '../Tweets'

/content/drive/My Drive/TCC/Tweets


In [56]:
def preprocessing(pd_serie):

  # Converting to lowercase
  pd_serie = pd_serie.str.lower()

  # Removing punctuation rules
  pd_serie = pd_serie.str.translate(str.maketrans('', '', string.punctuation))

  # Removing unused links
  pd_serie = pd_serie.str.replace(r"(http|@)\S+", "")

  # Transform short negation form
  pd_serie = pd_serie.str.replace(r"(nao| n | ñ )", 'não')

  # Remove special chars
  pd_serie = pd_serie.apply(demojize)
  pd_serie = pd_serie.str.replace(r"::", ": :")
  pd_serie = pd_serie.str.replace(r"’", "'")
  pd_serie = pd_serie.str.replace(r"[^a-z\':_]", " ")

  # Remove repetitions
  pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
  pd_serie = pd_serie.str.replace(pattern, r"\1")

  # Removing stop words
  sw = stopwords.words('portuguese')
  sw.remove('não')

  pd_serie = pd_serie.apply(
      lambda pd_serie: ' '.join([word for word in pd_serie.split() if word not in sw])
  )

  return pd_serie

**Add negation tag to make emphasis in a negative phrase**

In [57]:
def negative_phrase(phrase):
  negative_word = 'não'
  has_nagative_word = False
  new_phrase = []
  for word in phrase.split():
    if has_negative_word == True:
      word = word + '_NÃO'
    if word == negative_word:
      has_negative_word = True
    new_phrase.append(word)
  return (' '.join(new_phrase))

**Open stored tweets from CSV file**

In [58]:
df = pd.read_csv('train.csv')
df.head()

,created_at,user_id,screen_name,geo,coordinates,place,tweet_id,text,class
0,8/12/2020 23:59,1.240000e+18,lavnia57,NaN,NaN,NaN,1.290000e+18,Ia dar um chutao nela p sair da frente kkkkkkk...,0
1,8/12/2020 23:59,1.290000e+18,mendsbridgerton,NaN,NaN,NaN,1.290000e+18,Eu sempre te achei famosinha ai eu tinha medo ...,0
2,8/12/2020 23:59,1.060000e+18,jhe_yaya,NaN,NaN,NaN,1.290000e+18,RT @clatruvs: eu no banheiro da balada falando...,0
3,8/12/2020 23:59,1.270000e+18,Yasminpaivax,NaN,NaN,NaN,1.290000e+18,RT @clatruvs: eu no banheiro da balada falando...,0
4,8/12/2020 23:59,1.280000e+18,kopp_taliaa,NaN,NaN,NaN,1.290000e+18,Mano pq sair falando?? pqp,0


**Count lines in dataset**

In [59]:
df.text.count()

33726

**Removing duplicate lines**

In [60]:
df.drop_duplicates(['text'], inplace=True)

**Count lines**

In [61]:
df.text.count()

24948

**Preproccessing data**

In [62]:
classes = df['class']
tweets = df.text
tweets = preprocessing(tweets)
tweets

0                       ia dar chutao nela p sair frente k
1        sempre achei famosinha ai medo interagir vamos...
2        rt clatruvs banheiro balada falando pra menina...
4                                 mano pq sair falando pqp
5        doudementana entao vamos ficar separadas pq eu...
                               ...                        
33721    rt iasgomes favor doria fiel protocolo n tome ...
33722    mlkd ta td aq base vou clc beat p noix resenha...
33723    mafecustodio real to entendendo so to indo ta ...
33724    vou voltar malhar segunda aff vou ficar chata ...
33725    queria ta churrasco familia cantando molejo ma...
Name: text, Length: 24948, dtype: object

## **4. Training and predict process**

**Required libraries**

In [63]:
import sklearn
import joblib
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer

### **3.1 - MLPClassifier**

**Create a Multilayer Perceptron Model**

- Hidden layers = 1
- Neurons = 10
- Learning rate = 0.01
- Max iteration = 500
- Optimizer = Stochastic Gradient Descent with no batch-size

In [64]:
"""mlp_model = MLPClassifier(hidden_layer_sizes=(10), 
                          solver='sgd', 
                          learning_rate_init=0.01,
                          max_iter=500,
                          random_state=113)"""

mlp_model = MLPClassifier(learning_rate_init=0.01,
                          random_state=42)                

**Create pipeline for MLP classifier**

In [65]:
pipe = Pipeline([('vectorizer', CountVectorizer()), ('model', mlp_model)])

pipe_bigrams = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))), ('model', mlp_model)])

pipe_negation = Pipeline([
  ('vectorizer', CountVectorizer(tokenizer=lambda phrase: negative_phrase(phrase))), 
  ('model', mlp_model)
])

**Split x and y (feature and target)**

In [66]:
train_size=0.8
X_train, X_test, y_train, y_test = train_test_split(tweets,
                                                    classes,
                                                    train_size=train_size)

In [67]:
pd.DataFrame({'tweet': X_train, 'class': y_train})

,tweet,class
565,vou estrear filme mulher feia mundo,0
20253,vc quiser gente acorda antes sol nascer vc mul...,0
17331,vamos fazer outro galo,0
5788,carloscezarcjr stfvergonhamundial ira exigir d...,0
22412,homem quer acabar time qualquer maneira arrasc...,0
...,...,...
14093,rt paparazzorn domenec torrent desembarcando r...,0
13535,ment sbtw neymarjr serio voce ja pra balada ne...,0
17019,ah n vou descer xingo nela agora,0
17420,referencia marcelinho lendo contos eroticos ag...,0


In [68]:
pd.DataFrame({'tweet': X_test, 'class': y_test})

,tweet,class
6155,gente antiga starlanterns segue nessa conra ai...,0
13614,rafinha realmente sair fudeu vezkk,0
7381,n vou fala nada pq gremio k joga contra ceara ...,0
23925,vamos flamengo crlh,0
24251,rt neymarjr p alguem ai quer ser divulgado com...,0
...,...,...
432,saindo academia vim parar aniversario crianca ...,1
5316,mutualloveoned tbm acho porem pobre pra compra...,0
2686,rt barbarasousa vamos praia ver sol comer pizza,1
28721,acabei ter inspiracao pra festa acontecer ne,0


**Run training MLP model**

In [70]:
pipe.fit(X_train, y_train)

score = pipe.score(X_test, y_test)
print('\r{} [INFO] Score of MLP model with testing data is {:.1%}\n'
  .format(dt.datetime.now(), (1-train_size), score))

2020-10-04 13:52:24.325200 [INFO] Score of MLP model with testing data is 20.0%



**Cross validation**

In [71]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [72]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = cross_validate(pipe, X_train, y_train, cv=kfold)
print("Average accuracy: %f (%f)" %(results['test_score'].mean(), results['test_score'].std()))

results = cross_val_predict(pipe, X_train, y_train, cv=kfold)
metrics.accuracy_score(y_train, results)

Average accuracy: 0.967331 (0.001659)


0.967331395931456

In [73]:
print(metrics.classification_report(y_train, results, pipe.classes_))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     19375
           1       0.40      0.23      0.29       583

    accuracy                           0.97     19958
   macro avg       0.69      0.61      0.64     19958
weighted avg       0.96      0.97      0.96     19958



**Confusion matrix**

In [74]:
print(pd.crosstab(y_train, results, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted      0    1    All
Real                        
0          19174  201  19375
1            451  132    583
All        19625  333  19958


**Store MPL model in disk**

In [ ]:
file_name = 'mlp_model'
joblib.dump(pipe, file_name)
print('\rMPL model was saved sucessfully!')

MPL model was saved sucessfully!


**Load MLP model**

In [ ]:
pipe = joblib.load(file_name)

**Predict tweets**

In [75]:
tests = X_test
predict_result = pipe.predict(X_test)

pd.DataFrame(zip(tests, predict_result), columns=['tweet', 'class'])

,tweet,class
0,gente antiga starlanterns segue nessa conra ai...,0
1,rafinha realmente sair fudeu vezkk,0
2,n vou fala nada pq gremio k joga contra ceara ...,0
3,vamos flamengo crlh,0
4,rt neymarjr p alguem ai quer ser divulgado com...,0
...,...,...
4985,saindo academia vim parar aniversario crianca ...,1
4986,mutualloveoned tbm acho porem pobre pra compra...,0
4987,rt barbarasousa vamos praia ver sol comer pizza,1
4988,acabei ter inspiracao pra festa acontecer ne,0


**Prob for each class**

In [76]:
print(pipe.classes_)
pipe.predict_proba(X_test)

[0 1]


array([[9.99963274e-01, 3.67256107e-05],
       [9.99999776e-01, 2.23520385e-07],
       [1.00000000e+00, 7.55476010e-12],
       ...,
       [5.43545902e-02, 9.45645410e-01],
       [9.29623998e-01, 7.03760023e-02],
       [9.99999448e-01, 5.52248436e-07]])

### **3.2 - Naive Bayes**

### **3.3 - Sequential Minimal Optimization**